In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competition/training_set_features.csv
/kaggle/input/competition/test_set_features.csv
/kaggle/input/competition/training_set_labels.csv
/kaggle/input/competition/submission_format.csv


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer

# Load features and targets
features = pd.read_csv('/kaggle/input/competition/training_set_features.csv')
targets = pd.read_csv('/kaggle/input/competition/training_set_labels.csv')

# Merge datasets on respondent_id
data = pd.merge(features, targets, on='respondent_id')

# Separate features and target variables
X = data.drop(columns=['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'])
y = data[['xyz_vaccine', 'seasonal_vaccine']]

# Identify numerical and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Preprocessing pipeline for numerical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                    # Standardize numerical features
])

# Preprocessing pipeline for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode categorical features
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Define the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', MultiOutputClassifier(GradientBoostingClassifier(random_state=42)))
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict probabilities
y_pred_proba = model.predict_proba(X_test)
y_pred_proba = np.array([proba[:, 1] for proba in y_pred_proba]).T  # Get probabilities for class 1

# Evaluate the model
roc_auc = roc_auc_score(y_test, y_pred_proba, average='macro')
print('ROC AUC score:', roc_auc)

# Prepare test data for submission
test_data = pd.read_csv('/kaggle/input/competition/test_set_features.csv')

# Apply the same preprocessing steps to the test data
test_data[numeric_features] = test_data[numeric_features].fillna(test_data[numeric_features].median())
test_data[categorical_features] = test_data[categorical_features].fillna(test_data[categorical_features].mode().iloc[0])

# Drop respondent_id from test data
X_submission = test_data.drop(columns=['respondent_id'])

# Predict probabilities for submission data
y_submission_proba = model.predict_proba(X_submission)
y_submission_proba = np.array([proba[:, 1] for proba in y_submission_proba]).T  # Get probabilities for class 1

# Prepare the submission dataframe
submission = pd.DataFrame({
    'respondent_id': test_data['respondent_id'],
    'xyz_vaccine': y_submission_proba[:, 0],
    'seasonal_vaccine': y_submission_proba[:, 1]
})

# Save the submission to a CSV file
submission.to_csv('/kaggle/working/submission.csv', index=False)


ROC AUC score: 0.8499206587448243
